สิ่งที่ต้องเตรียมก่อน
ไฟล์จาก OMS report >> http://172.30.7.213/omsreport >> รายงาน >> กฟน.1 >> รายงานเหตุการณ์ไฟฟ้าขัดข้อง >> รายงานเหตุการไฟฟ้าขัดข้อง 2 เดือนล่าสุด 
save ข้อมูลลงไฟล์ OMS_N1_1, OMS_N1_2


In [14]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.4MB/s ta 0:00:011
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import numpy as np
import pandas as pd
import json
import re
import requests
import random
import neurolab as nl
from tqdm import tqdm
from time import sleep
from datetime import datetime

5

In [2]:
def decode(input):
    if input == 1:
        output = 9
    elif input == 2:
        output = 7
    elif input == 3:
        output = 5
    elif input == 4:
        output = 3
    elif input == 5:
        output = 1
    elif input == 6:
        output = 1/3
    elif input == 7:
        output = 1/5
    elif input == 8:
        output = 1/7
    else:
        output = 1/9
    return output

def count_APSA(df, feeder):
    temp_df=df.drop(set(np.where(df['feedername']!=feeder+'VB01')[0])).copy()
    print(temp_df)
    if temp_df['total'].vals==0:
        return {'percent_complete':0}
    else:
        return {'percent_complete':1-(temp_df['complete']/temp_df['total'])}


def count_counter(df, feeder):
    temp_df=df.drop(set(np.where(df['ฟีดเดอร์']!=feeder)[0])).copy()
    return {'T/R':list(temp_df['ประเภทการทำงาน']).count('T/R'), 'T/L':list(temp_df['ประเภทการทำงาน']).count('T/L')}

def nor_input_layer(df):
    df['sum_TR']=[item/10   if item < 10 else 1 for item in df['sum_TR']]
    df['sum_TL']=[item/10   if item < 10 else 1 for item in df['sum_TL']]
    df['Peak load']=[item/10   for item in df['Peak load']]
    df['Average load']=[item/10   for item in df['Average load']]
    return df



def get_max_load_N1(area, feeder, year,month):
    if area == 1:
        url = "http://172.30.7.209/services/get_load.php"

        querystring = {"region":"11","year":year,"month":month,"substation":feeder[:3],"feeder":"out{}".format(feeder[3:]),"type":"P"}
        headers = {
            'Cache-Control': "no-cache",
#             'Postman-Token': "eecdd298-920a-4726-91ba-f3f8d48ef2a8"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()
        if response['data'] != []:
            return float(response['max'])
        else:
            return float(0)

def get_ava_load_N1(area, feeder, year,month):
    if area == 1:
        url = "http://172.30.7.209/services/get_load.php"

        querystring = {"region":"11","year":year,"month":month,"substation":feeder[:3],"feeder":"out{}".format(feeder[3:]),"type":"P"}


        headers = {
            'Cache-Control': "no-cache",
#             'Postman-Token': "eecdd298-920a-4726-91ba-f3f8d48ef2a8"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()
    
        if response['data'] != []:
            return float(response['avg'])
        else:
            return float(0)
        
def get_max_load_N2(area, feeder, year,month):
    if area == 2:
        url = "http://172.30.7.209/services/get_load.php"

        querystring = {"region":"12","year":"{}".format(year),"month":"{}".format(month),"substaion":"{}".format(feeder[:3]),"feeder":"OUT{}".format(feeder[3:])}


        headers = {
            'Cache-Control': "no-cache",
#             'Postman-Token': "eecdd298-920a-4726-91ba-f3f8d48ef2a8"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()
        if response['data'] != []:
            return float(response['max'])
        else:
            return float(0)


def get_ava_load_N2(area, feeder, year,month):
    if area == 2:
        url = "http://172.30.7.209/services/get_load.php"

        querystring = {"region":"12","year":"{}".format(year),"month":"{}".format(month),"substaion":"{}".format(feeder[:3]),"feeder":"OUT{}".format(feeder[3:])}

        headers = {
            'Cache-Control': "no-cache",
#             'Postman-Token': "eecdd298-920a-4726-91ba-f3f8d48ef2a8"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()

        if response['data'] != []:
            return float(response['avg'])
        else:
            return float(0)

def get_peak_load_N3(area, feeder, year, month):
    if area == 3:
#         print(feeder)
        url = "http://172.30.200.113/webcenter/views/peakload.php"
        querystring = {"year":"{}".format(year),"month":"{}".format(month),"region":"13", 'feedername':'{}'.format(feeder)+'VB01'}
        headers = {
        'Cache-Control': "no-cache",
        #     'Postman-Token': "15f82df5-f82f-4596-bc0b-efd65dd23b28"
        }
        response = requests.request("GET", url, headers=headers, params=querystring)

        response=response.json()
        if response['PeakLoad'] == []:
            return float(0)
        else:
            return float(response['PeakLoad'][0]['MW'])
        
def clean_feeder(feeder):
    correct_feeder = ['OUT01', 'OUT02','OUT03','OUT04','OUT05','OUT06','OUT07','OUT08','OUT09','OUT10', 'OUT11','OUT12']
    if feeder not in correct_feeder:
        if feeder == 'OUT010':
            return 'OUT10'
        else:
            return 'OUT0'+feeder[3:]
    else:
        return feeder
    
def get_max_avarage_load(df, feeder):
    temp_df=df.drop(np.where(df['feeder']!=feeder)[0]).copy()
    return abs(temp_df['WERT_MAX'].max()), abs(temp_df['WERT_EFF'].mean())

def get_TL_TR_values(df1, df2, df3, List_feeder_name):
    
    df1=df1.drop(set(np.where(df1['ประเภทการทำงาน']=='D/F')[0]).union(set(np.where(df1['ประเภทการทำงาน']=='Operate')[0])))
    df1=df1.reset_index()
    del df1['index']
    del df1['หมายเลขเหตุการณ์']
    del df1['ลำดับ']
    del df1['วันที่/เวลาไฟดับ']
    del df1['เวลา']
    del df1['วันที่/เวลา ที่จ่ายไฟคืนระบบได้ครั้งแรก']
    del df1['วันที่/เวลา ที่จ่ายไฟคืนครบทั้งหมด']
    del df1['รวมเวลาไฟดับ (นาที)']
    del df1['เฟส']
    del df1['สาเหตุ/รายละเอียด']
    del df1['ทราบสาเหตุ']
    del df1['กฟฟ.รับผิดชอบ']
    del df1['สภาพอากาศ']
    del df1['ผชฟ. ถูกกระทบ (ราย)']
    del df1['สถานที่จุดเกิดเหตุ']
    del df1['รายละเอียดการแก้ไข']
    del df1['ค่าโหลด (MW)']
    del df1['ประเภทเหตุการณ์']

    df2=df2.drop(set(np.where(df2['ประเภทการทำงาน']=='D/F')[0]).union(set(np.where(df2['ประเภทการทำงาน']=='Operate')[0])))
    df2=df2.reset_index()
    del df2['index']
    del df2['หมายเลขเหตุการณ์']
    del df2['ลำดับ']
    del df2['วันที่/เวลาไฟดับ']
    del df2['เวลา']
    del df2['วันที่/เวลา ที่จ่ายไฟคืนระบบได้ครั้งแรก']
    del df2['วันที่/เวลา ที่จ่ายไฟคืนครบทั้งหมด']
    del df2['รวมเวลาไฟดับ (นาที)']
    del df2['เฟส']
    del df2['สาเหตุ/รายละเอียด']
    del df2['ทราบสาเหตุ']
    del df2['กฟฟ.รับผิดชอบ']
    del df2['สภาพอากาศ']
    del df2['ผชฟ. ถูกกระทบ (ราย)']
    del df2['สถานที่จุดเกิดเหตุ']
    del df2['รายละเอียดการแก้ไข']
    del df2['ค่าโหลด (MW)']
    del df2['ประเภทเหตุการณ์']
    
    df3=df3.drop(set(np.where(df3['ประเภทการทำงาน']=='D/F')[0]).union(set(np.where(df3['ประเภทการทำงาน']=='Operate')[0])))
    df3=df3.reset_index()
    del df3['index']
    del df3['หมายเลขเหตุการณ์']
    del df3['ลำดับ']
    del df3['วันที่/เวลาไฟดับ']
    del df3['เวลา']
    del df3['วันที่/เวลา ที่จ่ายไฟคืนระบบได้ครั้งแรก']
    del df3['วันที่/เวลา ที่จ่ายไฟคืนครบทั้งหมด']
    del df3['รวมเวลาไฟดับ (นาที)']
    del df3['เฟส']
    del df3['สาเหตุ/รายละเอียด']
    del df3['ทราบสาเหตุ']
    del df3['กฟฟ.รับผิดชอบ']
    del df3['สภาพอากาศ']
    del df3['ผชฟ. ถูกกระทบ (ราย)']
    del df3['สถานที่จุดเกิดเหตุ']
    del df3['รายละเอียดการแก้ไข']
    del df3['ค่าโหลด (MW)']
    del df3['ประเภทเหตุการณ์']
    
    df4=df1.append(df2)
    print(df4)
    df4=df4.reset_index()
    del df4['index']
    
    df4=df4.append(df3)
    print(df4)


    df4=df4.reset_index()
    del df4['index']
    
    return {feeder:count_counter(df4, feeder) for feeder in List_feeder_name}

# def get_OMS_data(region, month):
    

In [3]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [4]:
region = 1 #N
day = datetime.now().day
month = datetime.now().month
year = datetime.now().year

In [5]:
number_Criteria=10
dict_RI={1:0, 2:0, 3:0.58, 4:0.9, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.46, 10:1.5}
data=pd.read_csv('surveys.csv')
data=data.dropna()
data=data.reset_index()
df3=pd.DataFrame()
weights_list=['w1','w2','w3','w4','w5','w6','w7','w8','w9','w10']
df3=pd.DataFrame(columns=weights_list)
list(data.index)
CR_val=[]
dict={}
for index in list(data.index):
    input_file=list(data.iloc[index,:])
    prepared_file=[]
    for x in input_file:
        prepared_file.append(decode(x))
    columns_name=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
    #columns_name =['CB', 'RCS', 'Load Break', 'LRC', 'Recloser', 'SCB', 'SCB(FRTU)', 'FCB', 'LRR', 'AVR']
    #w1, w2, w4, w7
    df = pd.DataFrame(np.zeros([number_Criteria,number_Criteria]), columns=columns_name, index=columns_name)
    for name in columns_name:
        df[name][name]=1

        mini_list1=np.array(prepared_file[0:9].copy())  #9-0 =9
    df.iloc[0,1:10]=mini_list1
    df.iloc[1:10,0]=1/mini_list1

    mini_list2 = np.array(prepared_file[9:17].copy())   # 17-9 =8
    df.iloc[1,2:10]=mini_list2
    df.iloc[2:10,1]=1/mini_list2

    mini_list3 = np.array(prepared_file[17:24].copy())   #.  24-17 = 7
    df.iloc[2,3:10]=mini_list3
    df.iloc[3:10,2]=1/mini_list3

    mini_list4 = np.array(prepared_file[24:30].copy())   #.  30-24 = 6
    df.iloc[3,4:10]=mini_list4
    df.iloc[4:10,3]=1/mini_list4

    mini_list5 = np.array(prepared_file[30:35].copy())   #.  35-30 = 5
    df.iloc[4,5:10]=mini_list5
    df.iloc[5:10,4]=1/mini_list5

    mini_list6 = np.array(prepared_file[35:39].copy())   #.  39-35 = 4
    df.iloc[5,6:10]=mini_list6
    df.iloc[6:10,5]=1/mini_list6

    mini_list7 = np.array(prepared_file[39:42].copy())   #.  42-39 = 3
    df.iloc[6,7:10]=mini_list7
    df.iloc[7:10,6]=1/mini_list7

    mini_list8 = np.array(prepared_file[42:44].copy())   #.  44-42 = 2
    df.iloc[7,8:10]=mini_list8
    df.iloc[8:10,7]=1/mini_list8

    mini_list9 = np.array(prepared_file[44:45].copy())   #.  45-44 = 1
    df.iloc[8,9:10]=mini_list9
    df.iloc[9:10,8]=1/mini_list9
    
    df.loc['sum'] = df.sum()
    df2=pd.DataFrame(columns=columns_name, index=columns_name)
    for item in columns_name:
        df2.loc[item]=df.loc[item]/df.loc['sum']
    df2['total'] = df2[df2.columns].sum(axis=1)
    df2['weight'] = df2['total']/df2['total'].sum()
    df2.loc['sum'] = df2.sum()
    df3.loc[index]=np.array(list(df2['weight'][0:-1]))

    CI_list=[np.array(df.iloc[item,:].sum()*np.array(df2.iloc[0:10,11]))/df2.iloc[item,11] for item in np.arange(0,number_Criteria,1)]
    df2['CM']=np.array(CI_list+[np.mean(CI_list)])
    
    CI_val=(df2.iloc[10,12]-df2.iloc[10,10])/(df2.iloc[10,10]-1)
    CR_val.append(CI_val/dict_RI[number_Criteria])
    dict.update({index:{'weight':list(df2['weight']), 'CR':CI_val/dict_RI[number_Criteria]}})

df3['CR_val']=np.array(CR_val)

number_obs=56 #manual added
dict_weights={weight: np.prod(np.array(df3[weight]))**(1/number_obs) for weight in weights_list}
sum=dict_weights['w1']+dict_weights['w2']+dict_weights['w3']+dict_weights['w4']+dict_weights['w5']+dict_weights['w6']+dict_weights['w7']+dict_weights['w8']+dict_weights['w9']+dict_weights['w10']

#dict_nor_weights is output weight 
dict_nor_weights={weight:dict_weights[weight]/sum for weight in weights_list}
# print(dict_nor_weights)


pattern = '[A-Z]{3}\d\d'
rex=re.compile(pattern)

json1 = json.loads(open('n1.json').read())
List_N1_feeder_name= [rex.findall(dict['attributes']['FACILITYID'][0:5]) for dict in json1['features']]
List_N1_feeder_name=[a[0] for a in List_N1_feeder_name if a]

set_A=set([x for x in List_N1_feeder_name if List_N1_feeder_name.count(x)>1])
# print(set_A)
List_N1_feeder_name=list(set(List_N1_feeder_name))
set_A=set([x for x in List_N1_feeder_name if List_N1_feeder_name.count(x)>1])
# print(set_A)

List_N1_CB=[item + 'VB01' for item in List_N1_feeder_name]

json2 = json.loads(open('n2.json').read())
List_N2_feeder_name= [rex.findall(dict['attributes']['FACILITYID'][0:5]) for dict in json2['features']]
List_N2_feeder_name=[a[0] for a in List_N2_feeder_name if a]

set_A=set([x for x in List_N2_feeder_name if List_N2_feeder_name.count(x)>1])
# print(set_A)
List_N2_feeder_name=list(set(List_N2_feeder_name))
set_A=set([x for x in List_N2_feeder_name if List_N2_feeder_name.count(x)>1])
# print(set_A)

List_N2_CB=[item + 'VB01' for item in List_N2_feeder_name]

json3 = json.loads(open('n3.json').read())
List_N3_feeder_name= [rex.findall(dict['attributes']['FACILITYID'][0:5]) for dict in json3['features']]
List_N3_feeder_name=[a[0] for a in List_N3_feeder_name if a]
set_A=set([x for x in List_N3_feeder_name if List_N3_feeder_name.count(x)>1])
# print(set_A)
List_N3_feeder_name=list(set(List_N3_feeder_name))
set_A=set([x for x in List_N3_feeder_name if List_N3_feeder_name.count(x)>1])
# print(set_A)
List_N3_CB=[item + 'VB01' for item in List_N3_feeder_name]

In [6]:
for area in [1,2,3]:
    print('-------------start area {}-------------'.format(area))
    list_of_equipment_code=[10,11,14,16] #SCB, CB, Recloser, Switch
    if area == 1:
        all_list_feeder_name=List_N1_feeder_name
    elif area ==2:
         all_list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        all_list_feeder_name=List_N3_feeder_name

    list_feeder_name=[]
    list_FRTU_name=[]
    list_type=[]
    
    print('-------------retrieve GIS data of area {}-------------'.format(area))
    for equipment_code in list_of_equipment_code:
        print('GIS equiment code', equipment_code)
        for feeder_name in tqdm(all_list_feeder_name): 
            temp_list_feeder_name=[]
            temp_list_FRTU_name=[]
            temp_list_type=[]
            res1=requests.get('http://gisn{}.pea.co.th/arcgis/rest/services/PEA/MapServer/{}/query?where=1%3D1+AND+FEEDERID+%3D+%27{}%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=pjson'.format(area, equipment_code, feeder_name))
            res=res1.json()
            if len(res['features'])>0:
                temp_list_feeder_name=[feeder_name]*len(res['features'])
                temp_list_FRTU_name=[dict['attributes']['FACILITYID'] for dict in res['features']]
                if equipment_code == 10:
                    temp_list_type=['SCB']*len(res['features'])
                elif equipment_code == 11:
                    temp_list_type=['CB']*len(res['features'])
                elif equipment_code == 14:
                    temp_list_type=['Recloser']*len(res['features'])
                else:
                    temp_list_type=['Switch']*len(res['features'])
                list_feeder_name=list_feeder_name+temp_list_feeder_name
                list_FRTU_name=list_FRTU_name+temp_list_FRTU_name
                list_type=list_type+temp_list_type
    df_equipment=pd.DataFrame()
    df_equipment['feeder']=list_feeder_name
    df_equipment['equipment']=list_FRTU_name
    df_equipment['type']=list_type
    df_equipment['type'].unique()
    df_equipment.to_csv('equipment_{}.csv'.format(area), sep=',')
    # print(df_equipment)

    dict_feeder = {item1:{item2:[df_equipment.iloc[item3, 1] for item3 in set(np.where(df_equipment['type']==item2)[0]).intersection(np.where(df_equipment['feeder']==item1)[0])] for item2 in df_equipment['type'].unique()} for item1 in all_list_feeder_name}
    
    json_dict = json.dumps(dict_feeder)
    f = open("dict_area_{}.json".format(area),"w")
    f.write(json_dict)
    f.close()

    df_equipment=pd.read_csv('equipment_{}.csv'.format(area), sep=',')
    
    with open("dict_area_1.json", "r") as read_file:
        dict_feeder = json.load(read_file)

    #retieve APSA data
    print('-------------retrieve APSA data of area {}-------------'.format(area))
    res2=requests.get("https://region1.pea.co.th/api/apsa/status/2019")
    res2=res2.json()
    list_feedername=[dict['feedername'] for dict in res2['records']]
    list_finished=[int(dict['finished']) for dict in res2['records']]
    list_operation=[int(dict['operation']) for dict in res2['records']]
    df_APSA=pd.DataFrame()
    df_APSA['feedername']=list_feedername
    df_APSA['complete']=list_finished
    df_APSA['in_progress']=list_operation
    df_APSA['total']=df_APSA['in_progress']+df_APSA['complete']

    if area == 1:
        list_feeder=List_N1_CB
    elif area ==2:
         list_feeder=List_N2_CB
    elif area ==3:    
        list_feeder=List_N3_CB

    df2_APSA=pd.DataFrame()
    for feeder in tqdm(list_feeder):
        if feeder in set(df_APSA['feedername']):
            df2_APSA=df2_APSA.append(df_APSA.loc[df_APSA['feedername'].values==feeder])
        else:
            df_empty=pd.DataFrame()
            df_empty['feedername']=[feeder]
            df_empty['complete']=[0]
            df_empty['in_progress']=[0]
            df_empty['total']=[0]
            df2_APSA=df2_APSA.append(df_empty)
        
    df2_APSA=df2_APSA.reset_index()
    df2_APSA.to_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')
    
    df2_APSA=pd.read_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')
    df2_APSA['%in_complete']=1-df2_APSA['complete']/df2_APSA['total']
    df2_APSA=df2_APSA.fillna(0)
    # df_e_counter=pd.read_csv('e_counter_DataFrame_{}.csv'.format(area), sep=',')
    df_equipment=pd.read_csv('equipment_{}.csv'.format(area), sep=',')
    df_equipment= df_equipment.dropna()
    df_equipment = df_equipment.drop_duplicates(subset='equipment')
    df_equipment=df_equipment.reset_index()
    df_equipment=df_equipment.drop(df_equipment.index[[ind for ind in df_equipment.index if 'F-' in df_equipment.iloc[ind,3]]])
    df_equipment=df_equipment.reset_index()

    del df_equipment['level_0']
    del df_equipment['index']
    del df_equipment['Unnamed: 0']


    print('-------------retrieve OMS data of area {}-------------'.format(area))
#     if area == 1:
#         list_feeder_name=List_N1_feeder_name
#     elif area ==2:
#          list_feeder_name=List_N2_feeder_name
#     elif area ==3:    
#         list_feeder_name=List_N3_feeder_name
#     dict_feeder = {item1:{item2:[df_equipment.iloc[item3, 1] for item3 in set(np.where(df_equipment['type']==item2)[0]).intersection(np.where(df_equipment['feeder']==item1)[0])] for item2 in df_equipment['type'].unique()} for item1 in list_feeder_name}

    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
        list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name
    num_feeder=1
    dict_counter={}
    for feeder in tqdm(list_feeder_name):
#         print(feeder)
        num_feeder=num_feeder+1
        res2=requests.get("http://172.30.200.113/webcenter/views/omsp50.php?feeder={}&year={}".format(feeder, year))
        res2=res2.json()
        if res2 !=['{Not found DATA..}']:
            list_month_oms=[int(oms_detail['month'].replace(" ", "")) for oms_detail in res2]
            list_type_oms=[oms_detail['type_oper'] for oms_detail in res2]
            df_OMS=pd.DataFrame()
            df_OMS['month']=list_month_oms
            df_OMS['type']=list_type_oms
            temp_df_OMS=df_OMS.drop(np.where(df_OMS['month']!=month)[0]).copy()

            if 'T/L' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TL=temp_df_OMS['type'].value_counts()['T/L']
            else:
                num_TL=0
            if 'T/R' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TR=temp_df_OMS['type'].value_counts()['T/R']
            else:
                num_TR=0
        else:
    #             print('-- no OMS data --')
            num_TL, num_TR=0,0
    #         print(feeder , 'T/L:', num_TL, 'T/R:',num_TR)
        dict_counter.update({feeder:{'T/L':num_TL, 'T/R':num_TR}})
        pass
    print('-------------retrieve max, average load data of area {}-------------'.format(area))
    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
         list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name

    df_input_layer=pd.DataFrame()
    for feeder in tqdm(list_feeder_name):
        temp_df_input_layer=pd.DataFrame()
        input_layer1_2=df2_APSA.loc[df2_APSA['feedername'].values==feeder+'VB01']
        temp_df_input_layer['feeder']=[feeder]
        temp_df_input_layer['sum_TR']=dict_counter[feeder]['T/R']
        temp_df_input_layer['sum_TL']=dict_counter[feeder]['T/L']
        temp_df_input_layer['in_progress']=input_layer1_2['%in_complete'].values[0]
        
        if area == 1:
            temp_df_input_layer['Peak load']=abs(get_max_load_N1(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_ava_load_N1(area, feeder, year, month))
        elif area == 2:
            temp_df_input_layer['Peak load']=abs(get_max_load_N2(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_ava_load_N2(area, feeder, year, month))
        elif area == 3:
            temp_df_input_layer['Peak load']=abs(get_peak_load_N3(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_peak_load_N3(area, feeder, year, month)/4)
        df_input_layer=df_input_layer.append(temp_df_input_layer)
        pass
    
    df_input_layer=df_input_layer.reset_index()
    del df_input_layer['index']
    df_nor_input_layer=nor_input_layer(df_input_layer.copy())
    df_input_layer=df_input_layer.fillna(0)
    df_nor_input_layer=df_nor_input_layer.fillna(0)
          
        
        
    print('-------------FHM calculation of area {}-------------'.format(area))
    df_input_weight=pd.read_csv('weight_input.csv')
    del df_input_weight['Load Break']
    del df_input_weight['Recloser']
    del df_input_weight['SCB']
    del df_input_weight['Fixed Capacitor Bank']
    del df_input_weight['LRR(FRTU)']
    del df_input_weight['AVR']
    list_input=['sum_TR','sum_TL','in_progress','Peak load','Average load']

    input_range=[[0,1]]*(len(list(df_nor_input_layer))-1)  #[[x1_min, x1_max],[x2_min, x2_max],...,[xn_min, xn_max]]
    number_input=len(input_range)
    dict_FHI={}

    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
         list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name

    for feeder in tqdm(list_feeder_name):
    #     print(feeder)
        num_equipment = len(dict_feeder[feeder]['CB'])+len(dict_feeder[feeder]['SCB'])+len(dict_feeder[feeder]['Recloser'])+len(dict_feeder[feeder]['Switch'])
        num_CB=len(dict_feeder[feeder]['CB'])
        num_Switch=len(dict_feeder[feeder]['Switch'])
        num_Recloser=len(dict_feeder[feeder]['Recloser'])
        num_SCB=len(dict_feeder[feeder]['SCB'])
        if num_equipment !=0:
            list_equipment=[1]*num_CB + [2]*num_Switch + [3]*num_Recloser + [4]*num_SCB
            list_equipment_2=['w1']*num_CB + ['w2']*num_Switch + ['w4']*num_Recloser + ['w7']*num_SCB
            input_weight=np.array([[float(df_input_weight.iloc[np.where(df_input_weight['Input/Equipment']==input1)[0],equipment])for input1 in list_input] for equipment in list_equipment])
            nor_input_weight=np.array([input_weight[inp]/input_weight.sum(axis=1)[inp] for inp in np.arange(0,len(input_weight),1)])
            output_weight=np.array([dict_nor_weights[equipment]/{'w1':num_CB, 'w2':num_Switch, 'w4':num_Recloser, 'w7':num_SCB}[equipment] for equipment in list_equipment_2])
            nor_output_weight=output_weight/output_weight.sum(axis=0)
            net1 = nl.net.newff(input_range, [num_equipment, 1], [nl.trans.PureLin(), nl.trans.PureLin()])  #num_equipment =len [CB1, CB2, REC1, REC2, SW1, SW2]

            for input in np.arange(0,len(nor_input_weight),1):
                net1.layers[0].np['w'][input]= nor_input_weight[input]
            net1.layers[0].np['b']=[0]*num_equipment
            net1.layers[1].np['w'][0]=nor_output_weight
            net1.layers[0].np['b']=[0]*num_equipment
            net1.layers[1].np['b']=[0]
            df_nor_input_layer.iloc[np.where(df_nor_input_layer['feeder']==feeder)[0],1:]
            output=net1.sim(np.array(df_nor_input_layer.iloc[np.where(df_nor_input_layer['feeder']==feeder)[0],1:]).tolist())
    #         print(output)
            dict_FHI.update({feeder:1-output[0][0]})
        else:
            dict_FHI.update({feeder:1})


    # print(dict_FHI)
    df_input_layer['FHI']=[dict_FHI[feeder] for feeder in df_input_layer['feeder'].tolist()]
#     print(df_input_layer)

    with open("FHM_{}.json".format(are),'w') as outfile:
        json.dump(dict_FHI, outfile, cls=NumpyEncoder)
    
    print('-------------FHM area {} is saved-------------'.format(area))


print('-------------add urban area -------------')


#read all 3N json file
list_FHI=[]

list_urban={'1':['CMA01','CMA02','CMA03','CMA04','CMA05','CMA06','CMA07','CMA08','CMA09','CMA10','CMB01','CMB02','CMB03','CMB04','CMB05','CMB06','CMB07','CMB08','CMB09','CMB10','CMC01','CMC02','CMC03','CMD03','CMD07','CMD08','CMD10','CMF01','CMF02','CMF03','CMF04','CMF05','CMU01','CMU02','CMU03','CMU04','CMU05','CMV01','CMV02','CMV03','CMV04','CMV05','MRM07'],'2':['PLA02',
'PLA03','PLA04','PLA05','PLA06','PLA07','PLA08','PLC01','PLC02','PLC03','PLC04','PLC06','PLC07','PLT05','WGT05','MSA04','MSA05','MSA06','MSA11','MSA12'],'3':['LBA01','LBA02','LBA03','LBA04','LBA05','LBA06','LBA07','LBA08','LBC01','LBC02','LBC03','LBC04','LBC05']}
for area in tqdm([1,2,3]):
    with open('FHM_{}.json'.format(area)) as json_data:
        d=json.load(json_data)
        if area == 1:
            list_feeder_name=List_N1_feeder_name
        elif area ==2:
             list_feeder_name=List_N2_feeder_name
        elif area ==3:  
            print(d)
            list_feeder_name=List_N3_feeder_name
        list_FHI.append([{'Region':'1{}'.format(area), 'sub':feeder[0:3], 'feeder':feeder, 'HI':d[feeder], 'Urban':feeder in list_urban['{}'.format(area)]} for feeder in list_feeder_name])
        
all_dict_FHM={'raw_data':[item2 for item1 in list_FHI for item2 in item1]}

with open("FHM.json",'w') as outfile:
    json.dump(all_dict_FHM, outfile, cls=NumpyEncoder)
          
print('-------------FHM.json file is saved -------------')

-------------start area 1-------------
-------------retrieve GIS data of area 1-------------
10
NIC08
total 1/390
HCA05
total 2/390
CMA08
total 3/390
MCA07
total 4/390
MHA10
total 5/390
LPA10
total 6/390
CSA04
total 7/390
FAA10
total 8/390
CRA08
total 9/390
FAA01
total 10/390
SGA08
total 11/390
LNC07
total 12/390
LPA02
total 13/390
MLA03
total 14/390
DKA02
total 15/390
NIC03
total 16/390
MCA06
total 17/390
MHA01
total 18/390
NIC10
total 19/390
BGA02
total 20/390
CDO05
total 21/390
SGA04
total 22/390
CRB04
total 23/390
CRA10
total 24/390
MEA09
total 25/390
PAN07
total 26/390
LPA07
total 27/390
CRB02
total 28/390
CRA01
total 29/390
CMC02
total 30/390
CRB03
total 31/390
PAS04
total 32/390
CRA03
total 33/390
SKP03
total 34/390
LNB03
total 35/390
NIC01
total 36/390
WPA03
total 37/390
CSA02
total 38/390
MRM04
total 39/390
MCA03
total 40/390
LIA01
total 41/390
NIC09
total 42/390
HOA07
total 43/390
LPC01
total 44/390
TNG08
total 45/390
PYB05
total 46/390
PAN05
total 47/390
CMV03
total 48/390
M

CRB04
total 23/390
CRA10
total 24/390
MEA09
total 25/390
PAN07
total 26/390
LPA07
total 27/390
CRB02
total 28/390
CRA01
total 29/390
CMC02
total 30/390
CRB03
total 31/390
PAS04
total 32/390
CRA03
total 33/390
SKP03
total 34/390
LNB03
total 35/390
NIC01
total 36/390
WPA03
total 37/390
CSA02
total 38/390
MRM04
total 39/390
MCA03
total 40/390
LIA01
total 41/390
NIC09
total 42/390
HOA07
total 43/390
LPC01
total 44/390
TNG08
total 45/390
PYB05
total 46/390
PAN05
total 47/390
CMV03
total 48/390
MCA09
total 49/390
NIA08
total 50/390
CDO04
total 51/390
TNG02
total 52/390
CEA02
total 53/390
CMC10
total 54/390
DKA03
total 55/390
CME03
total 56/390
PYA08
total 57/390
WPA10
total 58/390
SKP04
total 59/390
PYA03
total 60/390
CMB09
total 61/390
CMF05
total 62/390
PAU04
total 63/390
MEA04
total 64/390
MRM10
total 65/390
SKP06
total 66/390
MLA05
total 67/390
CEA05
total 68/390
TNG03
total 69/390
NIU03
total 70/390
CMA04
total 71/390
PAN04
total 72/390
LNA05
total 73/390
SKP05
total 74/390
SGA07
total 

NIA08
total 50/390
CDO04
total 51/390
TNG02
total 52/390
CEA02
total 53/390
CMC10
total 54/390
DKA03
total 55/390
CME03
total 56/390
PYA08
total 57/390
WPA10
total 58/390
SKP04
total 59/390
PYA03
total 60/390
CMB09
total 61/390
CMF05
total 62/390
PAU04
total 63/390
MEA04
total 64/390
MRM10
total 65/390
SKP06
total 66/390
MLA05
total 67/390
CEA05
total 68/390
TNG03
total 69/390
NIU03
total 70/390
CMA04
total 71/390
PAN04
total 72/390
LNA05
total 73/390
SKP05
total 74/390
SGA07
total 75/390
MEA03
total 76/390
MRM02
total 77/390
DKA04
total 78/390
CSA05
total 79/390
HOA04
total 80/390
CMU02
total 81/390
MLA06
total 82/390
DKA10
total 83/390
SKP10
total 84/390
NIA05
total 85/390
MRM07
total 86/390
MLA04
total 87/390
MCA05
total 88/390
PAN10
total 89/390
NIA04
total 90/390
WPA02
total 91/390
CMA10
total 92/390
LNA01
total 93/390
CRB06
total 94/390
LPC04
total 95/390
PAN01
total 96/390
CHM04
total 97/390
CUN07
total 98/390
CKN04
total 99/390
CMV01
total 100/390
MCA01
total 101/390
MHA06
tota

MRM02
total 77/390
DKA04
total 78/390
CSA05
total 79/390
HOA04
total 80/390
CMU02
total 81/390
MLA06
total 82/390
DKA10
total 83/390
SKP10
total 84/390
NIA05
total 85/390
MRM07
total 86/390
MLA04
total 87/390
MCA05
total 88/390
PAN10
total 89/390
NIA04
total 90/390
WPA02
total 91/390
CMA10
total 92/390
LNA01
total 93/390
CRB06
total 94/390
LPC04
total 95/390
PAN01
total 96/390
CHM04
total 97/390
CUN07
total 98/390
CKN04
total 99/390
CMV01
total 100/390
MCA01
total 101/390
MHA06
total 102/390
NIU01
total 103/390
CMA02
total 104/390
CRA05
total 105/390
FAA03
total 106/390
LNB06
total 107/390
CRA06
total 108/390
SGA06
total 109/390
TEA03
total 110/390
PYB03
total 111/390
MHA02
total 112/390
CUN10
total 113/390
CHM03
total 114/390
MEA05
total 115/390
LPB05
total 116/390
WPA01
total 117/390
FAA06
total 118/390
PAS02
total 119/390
CRA09
total 120/390
LNC02
total 121/390
LNB08
total 122/390
CME02
total 123/390
LPA08
total 124/390
NIA01
total 125/390
CRA02
total 126/390
MRM09
total 127/390
PAU

KPA07
total 99/283
PIA02
total 100/283
TPH07
total 101/283
KPA02
total 102/283
UTA02
total 103/283
NAB02
total 104/283
RKA02
total 105/283
AKR02
total 106/283
WGT05
total 107/283
PRA07
total 108/283
MSU04
total 109/283
PLB01
total 110/283
KKU01
total 111/283
TSK02
total 112/283
TPH08
total 113/283
TAA03
total 114/283
PLC06
total 115/283
WBT05
total 116/283
TPH10
total 117/283
KPB04
total 118/283
TSK05
total 119/283
SSL01
total 120/283
WGT02
total 121/283
PIA09
total 122/283
KKU02
total 123/283
MSA12
total 124/283
MSU01
total 125/283
PIH01
total 126/283
RKA08
total 127/283
NAB05
total 128/283
MSA08
total 129/283
NAT03
total 130/283
PLA03
total 131/283
SKA01
total 132/283
WGT10
total 133/283
PKT01
total 134/283
PUU09
total 135/283
PUU02
total 136/283
RKA09
total 137/283
LKA02
total 138/283
KKU06
total 139/283
SKA04
total 140/283
KKU05
total 141/283
PLT05
total 142/283
STA01
total 143/283
MSA03
total 144/283
SSL04
total 145/283
DCA08
total 146/283
TPH02
total 147/283
RKA04
total 148/283
S

PPH03
total 231/283
TAA05
total 232/283
HTA01
total 233/283
PRA06
total 234/283
KKU08
total 235/283
NAA03
total 236/283
PUU01
total 237/283
PLA10
total 238/283
KKU04
total 239/283
PSN05
total 240/283
BNA02
total 241/283
PIH10
total 242/283
KPA09
total 243/283
SZA06
total 244/283
PLA05
total 245/283
PKT06
total 246/283
LKA03
total 247/283
PUU10
total 248/283
PIH03
total 249/283
PLA01
total 250/283
STA04
total 251/283
SZA02
total 252/283
SWA02
total 253/283
PRA04
total 254/283
NAA04
total 255/283
KKU03
total 256/283
KPA05
total 257/283
BNA05
total 258/283
UTA05
total 259/283
UTA07
total 260/283
KKU07
total 261/283
TAA08
total 262/283
KPA04
total 263/283
WGT06
total 264/283
DCA07
total 265/283
NAA07
total 266/283
TPH03
total 267/283
STA07
total 268/283
PLC02
total 269/283
BNA06
total 270/283
SKA03
total 271/283
NAT02
total 272/283
PLB04
total 273/283
WGT09
total 274/283
AKR01
total 275/283
DCA04
total 276/283
KPB01
total 277/283
WBA01
total 278/283
MSA02
total 279/283
TAA04
total 280/283


PLA02
total 85/283
KPB02
total 86/283
SSL05
total 87/283
SWA04
total 88/283
BHA02
total 89/283
STA09
total 90/283
PLT02
total 91/283
SZA08
total 92/283
NAB01
total 93/283
SWA06
total 94/283
RKA01
total 95/283
PLB05
total 96/283
KKU10
total 97/283
PLC05
total 98/283
KPA07
total 99/283
PIA02
total 100/283
TPH07
total 101/283
KPA02
total 102/283
UTA02
total 103/283
NAB02
total 104/283
RKA02
total 105/283
AKR02
total 106/283
WGT05
total 107/283
PRA07
total 108/283
MSU04
total 109/283
PLB01
total 110/283
KKU01
total 111/283
TSK02
total 112/283
TPH08
total 113/283
TAA03
total 114/283
PLC06
total 115/283
WBT05
total 116/283
TPH10
total 117/283
KPB04
total 118/283
TSK05
total 119/283
SSL01
total 120/283
WGT02
total 121/283
PIA09
total 122/283
KKU02
total 123/283
MSA12
total 124/283
MSU01
total 125/283
PIH01
total 126/283
RKA08
total 127/283
NAB05
total 128/283
MSA08
total 129/283
NAT03
total 130/283
PLA03
total 131/283
SKA01
total 132/283
WGT10
total 133/283
PKT01
total 134/283
PUU09
total 135

MRA04
total 213/269
LBC03
total 214/269
SBA10
total 215/269
KBG03
total 216/269
WIR02
total 217/269
BSH06
total 218/269
LBC01
total 219/269
NSA05
total 220/269
TCA05
total 221/269
NSA08
total 222/269
PRI02
total 223/269
KSR07
total 224/269
HKA04
total 225/269
BPI05
total 226/269
WIR01
total 227/269
BSH10
total 228/269
LAA10
total 229/269
NSU02
total 230/269
BQA09
total 231/269
NOB02
total 232/269
WIR05
total 233/269
TKB04
total 234/269
TKB05
total 235/269
BNA02
total 236/269
NSA04
total 237/269
SZA06
total 238/269
UAA04
total 239/269
NCH02
total 240/269
CNA03
total 241/269
NSA10
total 242/269
LBA02
total 243/269
UAA07
total 244/269
SZA02
total 245/269
IBU03
total 246/269
LOA01
total 247/269
LBA03
total 248/269
CNA06
total 249/269
PXA10
total 250/269
CYA02
total 251/269
BQA07
total 252/269
UAA09
total 253/269
LAA01
total 254/269
PEA05
total 255/269
TKB02
total 256/269
CDB04
total 257/269
PRI06
total 258/269
HKA10
total 259/269
WAM07
total 260/269
TCA03
total 261/269
HKA07
total 262/269


NOB05
total 95/269
HKA06
total 96/269
TKB07
total 97/269
LBB04
total 98/269
KBG02
total 99/269
PXA06
total 100/269
BSH08
total 101/269
CNA10
total 102/269
CNA08
total 103/269
CDB07
total 104/269
NSB03
total 105/269
MRA03
total 106/269
CNA02
total 107/269
SBA01
total 108/269
NOB03
total 109/269
BSH07
total 110/269
MRA08
total 111/269
NCH07
total 112/269
LBB02
total 113/269
NSA07
total 114/269
LAA04
total 115/269
TKB01
total 116/269
NSB09
total 117/269
BQA08
total 118/269
IBU01
total 119/269
TCA02
total 120/269
PRI05
total 121/269
SBA09
total 122/269
SBA05
total 123/269
LBB05
total 124/269
DSA05
total 125/269
IBU05
total 126/269
HKA08
total 127/269
SBA02
total 128/269
LOA04
total 129/269
WAM01
total 130/269
UAA05
total 131/269
CNA07
total 132/269
LOA05
total 133/269
BSH02
total 134/269
CDA06
total 135/269
UAA06
total 136/269
MRA01
total 137/269
ATB06
total 138/269
PXA07
total 139/269
PRI07
total 140/269
BSH01
total 141/269
CNA05
total 142/269
PRI01
total 143/269
PSN03
total 144/269
CDB02

NSA10
total 242/269
LBA02
total 243/269
UAA07
total 244/269
SZA02
total 245/269
IBU03
total 246/269
LOA01
total 247/269
LBA03
total 248/269
CNA06
total 249/269
PXA10
total 250/269
CYA02
total 251/269
BQA07
total 252/269
UAA09
total 253/269
LAA01
total 254/269
PEA05
total 255/269
TKB02
total 256/269
CDB04
total 257/269
PRI06
total 258/269
HKA10
total 259/269
WAM07
total 260/269
TCA03
total 261/269
HKA07
total 262/269
CON02
total 263/269
CDB06
total 264/269
WAM02
total 265/269
LBB01
total 266/269
KSR05
total 267/269
LOK05
total 268/269
LAA07
total 269/269


# APSA

In [5]:
res2=requests.get("https://region1.pea.co.th/api/apsa/status/2019")
res2=res2.json()
list_feedername=[dict['feedername'] for dict in res2['records']]
list_finished=[int(dict['finished']) for dict in res2['records']]
list_operation=[int(dict['operation']) for dict in res2['records']]
df_APSA=pd.DataFrame()
df_APSA['feedername']=list_feedername
df_APSA['complete']=list_finished
df_APSA['in_progress']=list_operation
df_APSA['total']=df_APSA['in_progress']+df_APSA['complete']

In [12]:
df_APSA.head()

,feedername,complete,in_progress,total
0,KHL05VB01,0,1,1
1,KHL06VB01,0,1,1
2,KHL09VB01,0,1,1
3,KLJ01YB01,1,0,1
4,KLJ01VR01,6,0,6


In [16]:
area=3
if area == 1:
    list_feeder=List_N1_CB
elif area ==2:
     list_feeder=List_N2_CB
elif area ==3:    
    list_feeder=List_N3_CB

df2_APSA=pd.DataFrame()
for feeder in list_feeder:
    if feeder in set(df_APSA['feedername']):
        df2_APSA=df2_APSA.append(df_APSA.loc[df_APSA['feedername'].values==feeder])
    else:
        df_empty=pd.DataFrame()
        df_empty['feedername']=[feeder]
        df_empty['complete']=[0]
        df_empty['in_progress']=[0]
        df_empty['total']=[0]
        df2_APSA=df2_APSA.append(df_empty)

df2_APSA=df2_APSA.reset_index()
df2_APSA.to_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')

df2_APSA=pd.read_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')
df2_APSA['%in_complete']=1-df2_APSA['complete']/df2_APSA['total']
df2_APSA=df2_APSA.fillna(0)
# df_e_counter=pd.read_csv('e_counter_DataFrame_{}.csv'.format(area), sep=',')
df_equipment=pd.read_csv('equipment_{}.csv'.format(area), sep=',')
df_equipment= df_equipment.dropna()
df_equipment = df_equipment.drop_duplicates(subset='equipment')
df_equipment=df_equipment.reset_index()
df_equipment=df_equipment.drop(df_equipment.index[[ind for ind in df_equipment.index if 'F-' in df_equipment.iloc[ind,3]]])
df_equipment=df_equipment.reset_index()

del df_equipment['level_0']
del df_equipment['index']
del df_equipment['Unnamed: 0']

In [17]:
df2_APSA.head()

,Unnamed: 0,index,feedername,complete,in_progress,total,%in_complete
0,0,0,NCH08VB01,0,0,0,0.000000
1,1,0,PXA08VB01,0,0,0,0.000000
2,2,1816,SBA07VB01,7,0,7,0.000000
3,3,1736,LAA03VB01,0,97,97,1.000000
4,4,1752,LOA02VB01,1,21,22,0.954545
5,5,0,HKA03VB01,0,0,0,0.000000
6,6,1710,CNA03VB01,51,7,58,0.120690
7,7,0,BQA10VB01,0,0,0,0.000000
8,8,1746,LBB01VB01,2,0,2,0.000000
9,9,1811,SBA02VB01,11,0,11,0.000000


# OMS

In [127]:
for area in [1,2,3]:
    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
        list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name
    num_feeder=1
    dict_counter={}
    for feeder in list_feeder_name:
#         print(feeder)
        print('{:.2f}%'.format(num_feeder/len(list_feeder_name)*100))
        num_feeder=num_feeder+1
        res2=requests.get("http://172.30.200.113/webcenter/views/omsp50.php?feeder={}&year={}".format(feeder, year))
        res2=res2.json()
        if res2 !=['{Not found DATA..}']:
            list_month_oms=[int(oms_detail['month'].replace(" ", "")) for oms_detail in res2]
            list_type_oms=[oms_detail['type_oper'] for oms_detail in res2]
            df_OMS=pd.DataFrame()
            df_OMS['month']=list_month_oms
            df_OMS['type']=list_type_oms
            temp_df_OMS=df_OMS.drop(np.where(df_OMS['month']!=month)[0]).copy()

            if 'T/L' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TL=temp_df_OMS['type'].value_counts()['T/L']
            else:
                num_TL=0
            if 'T/R' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TR=temp_df_OMS['type'].value_counts()['T/R']
            else:
                num_TR=0
        else:
    #             print('-- no OMS data --')
            num_TL, num_TR=0,0
    #         print(feeder , 'T/L:', num_TL, 'T/R:',num_TR)
        dict_counter.update({feeder:{'T/L':num_TL, 'T/R':num_TR}})
print(dict_counter)


0.26%
0.51%
0.77%
1.03%
1.28%
1.54%
1.79%
2.05%
2.31%
2.56%
2.82%
3.08%
3.33%
3.59%
3.85%
4.10%
4.36%
4.62%
4.87%
5.13%
5.38%
5.64%
5.90%
6.15%
6.41%
6.67%
6.92%
7.18%
7.44%
7.69%
7.95%
8.21%
8.46%
8.72%
8.97%
9.23%
9.49%
9.74%
10.00%
10.26%
10.51%
10.77%
11.03%
11.28%
11.54%
11.79%
12.05%
12.31%
12.56%
12.82%
13.08%
13.33%
13.59%
13.85%
14.10%
14.36%
14.62%
14.87%
15.13%
15.38%
15.64%
15.90%
16.15%
16.41%
16.67%
16.92%
17.18%
17.44%
17.69%
17.95%
18.21%
18.46%
18.72%
18.97%
19.23%
19.49%
19.74%
20.00%
20.26%
20.51%
20.77%
21.03%
21.28%
21.54%
21.79%
22.05%
22.31%
22.56%
22.82%
23.08%
23.33%
23.59%
23.85%
24.10%
24.36%
24.62%
24.87%
25.13%
25.38%
25.64%
25.90%
26.15%
26.41%
26.67%
26.92%
27.18%
27.44%
27.69%
27.95%
28.21%
28.46%
28.72%
28.97%
29.23%
29.49%
29.74%
30.00%
30.26%
30.51%
30.77%
31.03%
31.28%
31.54%
31.79%
32.05%
32.31%
32.56%
32.82%
33.08%
33.33%
33.59%
33.85%
34.10%
34.36%
34.62%
34.87%
35.13%
35.38%
35.64%
35.90%
36.15%
36.41%
36.67%
36.92%
37.18%
37.44%
37.69%
37.95%
38

In [ ]:
text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), progress*100, status)


In [128]:
x=[y for y in np.arange(0,100,1)]
barLength=20
num_x=1
for value in x:
    
    block=int(round(num_x/len(x)))
    if barLength % block
    print(block)
    num_x=num_x+1

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0


['{Not found DATA..}']

D/F        25
T/L        16
Operate    15
T/R         5
TR1         4
Name: type, dtype: int64

D/F        25
T/L        16
Operate    15
T/R         5
TR1         4
Name: type, dtype: int64

# SCADA

In [31]:
year=2018
month=2
feeder="NSB03"
area=3
load=get_peak_load_N3(area, feeder, year, month)
print(load)

0.0


In [ ]:
#

In [131]:
year=2019
month=1
for area in [1,2,3]:
    print('area N{}'.format(area))
    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
         list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name
    for feeder in list_feeder_name:
        print('feeder name:', feeder)
        if area == 1:
            print('Max load:', get_max_load_N1(area, feeder, year, month))
            print('Average load:', get_ava_load_N1(area, feeder, year, month))
        elif area == 2:
            print('Max load:', get_max_load_N2(area, feeder, year, month))
            print('Average load:', get_ava_load_N2(area, feeder, year, month))
        elif area == 3:
            print('Max load:', get_peak_load_N3(area, feeder, year, month))
            print('Average load:', get_peak_load_N3(area, feeder, year, month)/4)




area N1
feeder name: LPC03


ConnectionError: HTTPConnectionPool(host='172.30.7.209', port=80): Max retries exceeded with url: /services/get_load.php?region=11&year=2019&month=1&substation=LPC&feeder=out03&type=P (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x111027b70>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [132]:
res2=requests.get("http://172.30.7.209/services/get_load.php?region=11&year=2019&month=4&substation=LNB&feeder=out01&type=P")
res2=res2.json()



ConnectionError: HTTPConnectionPool(host='172.30.7.209', port=80): Max retries exceeded with url: /services/get_load.php?region=11&year=2019&month=4&substation=LNB&feeder=out01&type=P (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x110c66860>: Failed to establish a new connection: [Errno 61] Connection refused',))

# already got GIS data


In [6]:
for area in [1,2,3]:
    print('-------------start area {}-------------'.format(area))
    list_of_equipment_code=[10,11,14,16] #SCB, CB, Recloser, Switch
    if area == 1:
        all_list_feeder_name=List_N1_feeder_name
    elif area ==2:
         all_list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        all_list_feeder_name=List_N3_feeder_name

    list_feeder_name=[]
    list_FRTU_name=[]
    list_type=[]
    
    print('-------------retrieve GIS data of area {}-------------'.format(area))
#     for equipment_code in list_of_equipment_code:
#         print(equipment_code)
#         num=0
#         for feeder_name in all_list_feeder_name: 
#             print(feeder_name)
#             num=num+1
#             print('total {}/{}'.format(num, len(all_list_feeder_name)))
#             temp_list_feeder_name=[]
#             temp_list_FRTU_name=[]
#             temp_list_type=[]
#             res1=requests.get('http://gisn{}.pea.co.th/arcgis/rest/services/PEA/MapServer/{}/query?where=1%3D1+AND+FEEDERID+%3D+%27{}%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=pjson'.format(area, equipment_code, feeder_name))
#             res=res1.json()
#             if len(res['features'])>0:
#                 temp_list_feeder_name=[feeder_name]*len(res['features'])
#                 temp_list_FRTU_name=[dict['attributes']['FACILITYID'] for dict in res['features']]
#                 if equipment_code == 10:
#                     temp_list_type=['SCB']*len(res['features'])
#                 elif equipment_code == 11:
#                     temp_list_type=['CB']*len(res['features'])
#                 elif equipment_code == 14:
#                     temp_list_type=['Recloser']*len(res['features'])
#                 else:
#                     temp_list_type=['Switch']*len(res['features'])
#                 list_feeder_name=list_feeder_name+temp_list_feeder_name
#                 list_FRTU_name=list_FRTU_name+temp_list_FRTU_name
#                 list_type=list_type+temp_list_type
#     df_equipment=pd.DataFrame()
#     df_equipment['feeder']=list_feeder_name
#     df_equipment['equipment']=list_FRTU_name
#     df_equipment['type']=list_type
#     df_equipment['type'].unique()
#     df_equipment.to_csv('equipment_{}.csv'.format(area), sep=',')
#     # print(df_equipment)

#     dict_feeder = {item1:{item2:[df_equipment.iloc[item3, 1] for item3 in set(np.where(df_equipment['type']==item2)[0]).intersection(np.where(df_equipment['feeder']==item1)[0])] for item2 in df_equipment['type'].unique()} for item1 in all_list_feeder_name}
    
#     json_dict = json.dumps(dict_feeder)
#     f = open("dict_area_{}.json".format(area),"w")
#     f.write(json_dict)
#     f.close()

    df_equipment=pd.read_csv('equipment_{}.csv'.format(area), sep=',')
    
    with open("dict_area_1.json", "r") as read_file:
        dict_feeder = json.load(read_file)

    #retieve APSA data
    print('-------------retrieve APSA data of area {}-------------'.format(area))
    res2=requests.get("https://region1.pea.co.th/api/apsa/status/2019")
    res2=res2.json()
    list_feedername=[dict['feedername'] for dict in res2['records']]
    list_finished=[int(dict['finished']) for dict in res2['records']]
    list_operation=[int(dict['operation']) for dict in res2['records']]
    df_APSA=pd.DataFrame()
    df_APSA['feedername']=list_feedername
    df_APSA['complete']=list_finished
    df_APSA['in_progress']=list_operation
    df_APSA['total']=df_APSA['in_progress']+df_APSA['complete']

    if area == 1:
        list_feeder=List_N1_CB
    elif area ==2:
         list_feeder=List_N2_CB
    elif area ==3:    
        list_feeder=List_N3_CB

    df2_APSA=pd.DataFrame()
    for feeder in tqdm(list_feeder):
        if feeder in set(df_APSA['feedername']):
            df2_APSA=df2_APSA.append(df_APSA.loc[df_APSA['feedername'].values==feeder])
        else:
            df_empty=pd.DataFrame()
            df_empty['feedername']=[feeder]
            df_empty['complete']=[0]
            df_empty['in_progress']=[0]
            df_empty['total']=[0]
            df2_APSA=df2_APSA.append(df_empty)
        
    df2_APSA=df2_APSA.reset_index()
    df2_APSA.to_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')
    
    df2_APSA=pd.read_csv('APSA_DataFrame_{}.csv'.format(area), sep=',')
    df2_APSA['%in_complete']=1-df2_APSA['complete']/df2_APSA['total']
    df2_APSA=df2_APSA.fillna(0)
    # df_e_counter=pd.read_csv('e_counter_DataFrame_{}.csv'.format(area), sep=',')
    df_equipment=pd.read_csv('equipment_{}.csv'.format(area), sep=',')
    df_equipment= df_equipment.dropna()
    df_equipment = df_equipment.drop_duplicates(subset='equipment')
    df_equipment=df_equipment.reset_index()
    df_equipment=df_equipment.drop(df_equipment.index[[ind for ind in df_equipment.index if 'F-' in df_equipment.iloc[ind,3]]])
    df_equipment=df_equipment.reset_index()

    del df_equipment['level_0']
    del df_equipment['index']
    del df_equipment['Unnamed: 0']


    print('-------------retrieve OMS data of area {}-------------'.format(area))
#     if area == 1:
#         list_feeder_name=List_N1_feeder_name
#     elif area ==2:
#          list_feeder_name=List_N2_feeder_name
#     elif area ==3:    
#         list_feeder_name=List_N3_feeder_name
#     dict_feeder = {item1:{item2:[df_equipment.iloc[item3, 1] for item3 in set(np.where(df_equipment['type']==item2)[0]).intersection(np.where(df_equipment['feeder']==item1)[0])] for item2 in df_equipment['type'].unique()} for item1 in list_feeder_name}

    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
        list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name
    num_feeder=1
    dict_counter={}
    for feeder in tqdm(list_feeder_name):
#         print(feeder)
        num_feeder=num_feeder+1
        res2=requests.get("http://172.30.200.113/webcenter/views/omsp50.php?feeder={}&year={}".format(feeder, year))
        res2=res2.json()
        if res2 !=['{Not found DATA..}']:
            list_month_oms=[int(oms_detail['month'].replace(" ", "")) for oms_detail in res2]
            list_type_oms=[oms_detail['type_oper'] for oms_detail in res2]
            df_OMS=pd.DataFrame()
            df_OMS['month']=list_month_oms
            df_OMS['type']=list_type_oms
            temp_df_OMS=df_OMS.drop(np.where(df_OMS['month']!=month)[0]).copy()

            if 'T/L' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TL=temp_df_OMS['type'].value_counts()['T/L']
            else:
                num_TL=0
            if 'T/R' in list(temp_df_OMS['type'].value_counts().keys()):
                num_TR=temp_df_OMS['type'].value_counts()['T/R']
            else:
                num_TR=0
        else:
    #             print('-- no OMS data --')
            num_TL, num_TR=0,0
    #         print(feeder , 'T/L:', num_TL, 'T/R:',num_TR)
        dict_counter.update({feeder:{'T/L':num_TL, 'T/R':num_TR}})
        pass
    print('-------------retrieve max, average load data of area {}-------------'.format(area))
    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
         list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name

    df_input_layer=pd.DataFrame()
    for feeder in tqdm(list_feeder_name):
        temp_df_input_layer=pd.DataFrame()
        input_layer1_2=df2_APSA.loc[df2_APSA['feedername'].values==feeder+'VB01']
        temp_df_input_layer['feeder']=[feeder]
        temp_df_input_layer['sum_TR']=dict_counter[feeder]['T/R']
        temp_df_input_layer['sum_TL']=dict_counter[feeder]['T/L']
        temp_df_input_layer['in_progress']=input_layer1_2['%in_complete'].values[0]
        
        if area == 1:
            temp_df_input_layer['Peak load']=abs(get_max_load_N1(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_ava_load_N1(area, feeder, year, month))
        elif area == 2:
            temp_df_input_layer['Peak load']=abs(get_max_load_N2(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_ava_load_N2(area, feeder, year, month))
        elif area == 3:
            temp_df_input_layer['Peak load']=abs(get_peak_load_N3(area, feeder, year, month))
            temp_df_input_layer['Average load']=abs(get_peak_load_N3(area, feeder, year, month)/4)
        df_input_layer=df_input_layer.append(temp_df_input_layer)
        pass
    
    df_input_layer=df_input_layer.reset_index()
    del df_input_layer['index']
    df_nor_input_layer=nor_input_layer(df_input_layer.copy())
    df_input_layer=df_input_layer.fillna(0)
    df_nor_input_layer=df_nor_input_layer.fillna(0)
          
        
        
    print('-------------FHM calculation of area {}-------------'.format(area))
    df_input_weight=pd.read_csv('weight_input.csv')
    del df_input_weight['Load Break']
    del df_input_weight['Recloser']
    del df_input_weight['SCB']
    del df_input_weight['Fixed Capacitor Bank']
    del df_input_weight['LRR(FRTU)']
    del df_input_weight['AVR']
    list_input=['sum_TR','sum_TL','in_progress','Peak load','Average load']

    input_range=[[0,1]]*(len(list(df_nor_input_layer))-1)  #[[x1_min, x1_max],[x2_min, x2_max],...,[xn_min, xn_max]]
    number_input=len(input_range)
    dict_FHI={}

    if area == 1:
        list_feeder_name=List_N1_feeder_name
    elif area ==2:
         list_feeder_name=List_N2_feeder_name
    elif area ==3:    
        list_feeder_name=List_N3_feeder_name

    for feeder in list_feeder_name:
    #     print(feeder)
        num_equipment = len(dict_feeder[feeder]['CB'])+len(dict_feeder[feeder]['SCB'])+len(dict_feeder[feeder]['Recloser'])+len(dict_feeder[feeder]['Switch'])
        num_CB=len(dict_feeder[feeder]['CB'])
        num_Switch=len(dict_feeder[feeder]['Switch'])
        num_Recloser=len(dict_feeder[feeder]['Recloser'])
        num_SCB=len(dict_feeder[feeder]['SCB'])
        if num_equipment !=0:
            list_equipment=[1]*num_CB + [2]*num_Switch + [3]*num_Recloser + [4]*num_SCB
            list_equipment_2=['w1']*num_CB + ['w2']*num_Switch + ['w4']*num_Recloser + ['w7']*num_SCB
            input_weight=np.array([[float(df_input_weight.iloc[np.where(df_input_weight['Input/Equipment']==input1)[0],equipment])for input1 in list_input] for equipment in list_equipment])
            nor_input_weight=np.array([input_weight[inp]/input_weight.sum(axis=1)[inp] for inp in np.arange(0,len(input_weight),1)])
            output_weight=np.array([dict_nor_weights[equipment]/{'w1':num_CB, 'w2':num_Switch, 'w4':num_Recloser, 'w7':num_SCB}[equipment] for equipment in list_equipment_2])
            nor_output_weight=output_weight/output_weight.sum(axis=0)
            net1 = nl.net.newff(input_range, [num_equipment, 1], [nl.trans.PureLin(), nl.trans.PureLin()])  #num_equipment =len [CB1, CB2, REC1, REC2, SW1, SW2]

            for input in np.arange(0,len(nor_input_weight),1):
                net1.layers[0].np['w'][input]= nor_input_weight[input]
            net1.layers[0].np['b']=[0]*num_equipment
            net1.layers[1].np['w'][0]=nor_output_weight
            net1.layers[0].np['b']=[0]*num_equipment
            net1.layers[1].np['b']=[0]
            df_nor_input_layer.iloc[np.where(df_nor_input_layer['feeder']==feeder)[0],1:]
            output=net1.sim(np.array(df_nor_input_layer.iloc[np.where(df_nor_input_layer['feeder']==feeder)[0],1:]).tolist())
    #         print(output)
            dict_FHI.update({feeder:1-output[0][0]})
        else:
            dict_FHI.update({feeder:1})


    # print(dict_FHI)
    df_input_layer['FHI']=[dict_FHI[feeder] for feeder in df_input_layer['feeder'].tolist()]
    print(df_input_layer)

    with open("FHM_{}.json".format(are),'w') as outfile:
        json.dump(dict_FHI, outfile, cls=NumpyEncoder)
    
    print('-------------FHM area {} is saved-------------'.format(area))


print('-------------add urban area -------------')


#read all 3N json file
list_FHI=[]

list_urban={'1':['CMA01','CMA02','CMA03','CMA04','CMA05','CMA06','CMA07','CMA08','CMA09','CMA10','CMB01','CMB02','CMB03','CMB04','CMB05','CMB06','CMB07','CMB08','CMB09','CMB10','CMC01','CMC02','CMC03','CMD03','CMD07','CMD08','CMD10','CMF01','CMF02','CMF03','CMF04','CMF05','CMU01','CMU02','CMU03','CMU04','CMU05','CMV01','CMV02','CMV03','CMV04','CMV05','MRM07'],'2':['PLA02',
'PLA03','PLA04','PLA05','PLA06','PLA07','PLA08','PLC01','PLC02','PLC03','PLC04','PLC06','PLC07','PLT05','WGT05','MSA04','MSA05','MSA06','MSA11','MSA12'],'3':['LBA01','LBA02','LBA03','LBA04','LBA05','LBA06','LBA07','LBA08','LBC01','LBC02','LBC03','LBC04','LBC05']}
for area in [1,2,3]:
    with open('FHM_{}.json'.format(area)) as json_data:
        d=json.load(json_data)
        if area == 1:
            list_feeder_name=List_N1_feeder_name
        elif area ==2:
             list_feeder_name=List_N2_feeder_name
        elif area ==3:  
            print(d)
            list_feeder_name=List_N3_feeder_name
        list_FHI.append([{'Region':'1{}'.format(area), 'sub':feeder[0:3], 'feeder':feeder, 'HI':d[feeder], 'Urban':feeder in list_urban['{}'.format(area)]} for feeder in list_feeder_name])
        
all_dict_FHM={'raw_data':[item2 for item1 in list_FHI for item2 in item1]}

with open("FHM.json",'w') as outfile:
    json.dump(all_dict_FHM, outfile, cls=NumpyEncoder)
          
print('-------------FHM.json file is saved -------------')

-------------start area 1-------------
-------------retrieve GIS data of area 1-------------
-------------retrieve APSA data of area 1-------------


  0%|          | 0/390 [00:00<?, ?it/s]

-------------retrieve OMS data of area 1-------------


 22%|██▏       | 87/390 [01:49<05:10,  1.02s/it]

KeyboardInterrupt: 

In [136]:
dict_feeder

{'BGA01': {'CB': ['BGA01VR-01'],
  'Recloser': ['BGA01VR-101'],
  'SCB': ['BGA02C-02'],
  'Switch': ['BGA02F-07',
   'BGA01VS-105',
   'CEA03VS-105',
   'BGA01VF-172',
   'BGA01VF-171',
   'BGA02F-04',
   'BGA02VF-169',
   'BGA01VF-133',
   'BGA01VS-107',
   'BGA01VF-132',
   'BGA02VS-106',
   'BGA01VF-110',
   'BGA01VF-106',
   'BGA01VS-02',
   'BGA01VF-134',
   'BGA01F-10',
   'BGA01VF-131',
   'BGA01F-11',
   'BGA01VF-108',
   'BGA01VS-102',
   'BGA01VF-99',
   'BGA02F-06',
   'BGA01VF-101',
   'BGA01VS-104',
   'BGA02VF-170',
   'BGA01F-07',
   'BGA01VF-104',
   'BGA02F-03',
   'BGA01VF-107',
   'BGA02VF-168',
   'LIA03VS-109',
   'BGA02F-09',
   'BGA01VF-102',
   'BGA01VF-130',
   'BGA01F-16',
   'BGA02F-10',
   'BGA01VF-701',
   'BGA04VS-106',
   'BGA01VS-106',
   'BGA01F-12',
   'BGA02VF-173',
   'LIA01VS-108',
   'BGA02F-05',
   'BGA02VS-107',
   'BGA02VS-108',
   'BGA01VS-101',
   'BGA02VS-109',
   'BGA01VS-103',
   'BGA02F-08',
   'BGA01VF-702',
   'BGA01F-13',
   'BGA01VF-10

In [13]:
res1=requests.get('http://172.30.7.209/services/get_load.php?region=11&year=2019&month=04&substation=lnb&feeder=out01&type=P')

ConnectionError: HTTPConnectionPool(host='172.30.7.209', port=80): Max retries exceeded with url: /services/get_load.php?region=11&year=2019&month=04&substation=lnb&feeder=out01&type=P (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10f344470>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [19]:
from tqdm import tqdm
from time import sleep

In [20]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)
    



  0%|          | 0/4 [00:00<?, ?it/s]

Processing a:   0%|          | 0/4 [00:00<?, ?it/s]

Processing a:  25%|██▌       | 1/4 [00:00<00:00,  3.85it/s]

Processing b:  25%|██▌       | 1/4 [00:00<00:00,  3.85it/s]

Processing b:  50%|█████     | 2/4 [00:00<00:00,  3.85it/s]

Processing c:  50%|█████     | 2/4 [00:00<00:00,  3.85it/s]

Processing c:  75%|███████▌  | 3/4 [00:00<00:00,  3.85it/s]

Processing d:  75%|███████▌  | 3/4 [00:01<00:00,  3.85it/s]

Processing d: 100%|██████████| 4/4 [00:01<00:00,  3.83it/s]